In [1]:
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow import keras
import os 
import pandas as pd

In [2]:
# loading the csv metadata files
train_classes = pd.read_csv("../input/planets-dataset/planet/planet/train_classes.csv")
sample_sub = pd.read_csv("../input/planets-dataset/planet/planet/sample_submission.csv")
train_classes.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [3]:
sample_sub.head()

,image_name,tags
0,test_0,primary clear agriculture road water
1,test_1,primary clear agriculture road water
2,test_2,primary clear agriculture road water
3,test_3,primary clear agriculture road water
4,test_4,primary clear agriculture road water


In [4]:
# No of additional test images
len(os.listdir('../input/planets-dataset/test-jpg-additional/test-jpg-additional'))

20522

In [5]:
# No of test images
len(os.listdir('../input/planets-dataset/planet/planet/test-jpg'))

40669

In [6]:
# No of images to be submitted
len(sample_sub)

61191

In [7]:
import cv2
from tqdm import tqdm
import pandas as pd
import numpy as np

# loading the csv metadata files
train_classes = pd.read_csv("../input/planets-dataset/planet/planet/train_classes.csv")
sample_sub = pd.read_csv("../input/planets-dataset/planet/planet/sample_submission.csv")

label_map = {'agriculture': 0, 
             'artisinal_mine': 1, 
             'bare_ground': 2,
             'blooming': 3,
             'blow_down': 4,
             'clear': 5,
             'cloudy': 6,
             'conventional_mine': 7,
             'cultivation': 8,
             'habitation': 9,
             'haze': 10,
             'partly_cloudy': 11,
             'primary': 12,
             'road': 13, 
             'selective_logging': 14, 
             'slash_burn': 15, 
             'water': 16
             }

x_train, y_train = [], []
for img_name, tags in tqdm(train_classes.values, miniters=1000):
    arr = cv2.imread('../input/planets-dataset/planet/planet/train-jpg/{}.jpg'.format(img_name))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train.append(cv2.resize(arr, (90,90 )))
    y_train.append(targets)

    

# save the model and clear RAM by deleting x_train and y_train
from numpy import savez_compressed
savez_compressed('./planet_data.npz',x_train, y_train)
import gc
gc.collect()
del x_train, y_train

100%|██████████| 40479/40479 [02:14<00:00, 300.39it/s]


In [8]:
from numpy import load
data = load('./planet_data.npz')
x_train, y_train = data['arr_0'], data['arr_1']
print('Loaded: ', x_train.shape, y_train.shape)

from sklearn.model_selection import train_test_split


x_train, x_val, y_train, y_val = train_test_split(x_train,
                                                  y_train, 
                                                  test_size = 0.20, 
                                                  shuffle = True, 
                                                  random_state = 1)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

# calculate fbeta score for multi-class/label classification
from keras import backend

def fbeta(y_true, y_pred, beta=2):
	# clip predictions
	y_pred = backend.clip(y_pred, 0, 1)
	# calculate elements
	tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
	fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
	fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)
	# calculate precision
	p = tp / (tp + fp + backend.epsilon())
	# calculate recall
	r = tp / (tp + fn + backend.epsilon())
	# calculate fbeta, averaged across each class
	bb = beta ** 2
	fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
	return fbeta_score

# base model. Feel free to try out other architectures and ideas to improve fbeta score


from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D,MaxPool2D, BatchNormalization, Dropout
from keras.optimizers import Adam, RMSprop
from keras import Sequential
from keras.optimizers import Adam


in_shape=(90, 90, 3)
out_shape=17
model = Sequential()




model.add(Conv2D(input_shape=in_shape,filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same",kernel_initializer='he_uniform', activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same",kernel_initializer='he_uniform', activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same",kernel_initializer='he_uniform', activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same",kernel_initializer='he_uniform', activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same",kernel_initializer='he_uniform', activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same",kernel_initializer='he_uniform', activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same",kernel_initializer='he_uniform', activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same",kernel_initializer='he_uniform', activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same",kernel_initializer='he_uniform', activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same",kernel_initializer='he_uniform', activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same",kernel_initializer='he_uniform', activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=128,activation="relu"))
model.add(Dense(units=128,activation="relu"))
model.add(Dense(out_shape, activation='sigmoid'))

opt = Adam(lr=0.001)

# compile model
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[fbeta])

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1.0/255.0, 
                                   horizontal_flip=True, 
                                   vertical_flip=True, 
                                   rotation_range=90,
                                   height_shift_range=0.5,
                                   )
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
# prepare iterators
train_it = train_datagen.flow(x_train,
                              y_train,
                              batch_size=128)
test_it = test_datagen.flow(x_val, 
                            y_val, 
                            batch_size=128)
history = model.fit(train_it, 
                              steps_per_epoch=128,
                              validation_data=test_it, 
                              validation_steps=len(test_it), 
                              epochs=20, 
                              verbose=0)
history

Loaded:  (40479, 90, 90, 3) (40479, 17)
(32383, 90, 90, 3) (32383, 17) (8096, 90, 90, 3) (8096, 17)


In [16]:
# evaluate model
loss, fbeta = model.evaluate_generator(test_it,
                                       steps=len(test_it), 
                                       verbose=0)

print('> loss=%.3f, fbeta=%.3f' % (loss, fbeta))


> loss=0.128, fbeta=0.850


In [17]:
test_imgs=os.listdir('../input/planets-dataset/planet/planet/test-jpg')
test_imgs_add = os.listdir('../input/planets-dataset/test-jpg-additional/test-jpg-additional')

print(len(test_imgs))
print(len(test_imgs_add))
# for i in test_imgs:


40669
20522


In [11]:
test_data = test_imgs.extend(test_imgs_add)
print(len(test_imgs))
test_imgs = [v[:-4] for v in test_imgs]
test_df = pd.DataFrame(test_imgs,columns=['image_name']) 
test_df.head()

61191


,image_name
0,test_16476
1,test_15164
2,test_16984
3,test_4415
4,test_5227


In [18]:
submit_df = pd.read_csv('../input/planets-dataset/planet/planet/sample_submission.csv')
submit_df.head()

,image_name,tags
0,test_0,primary clear agriculture road water
1,test_1,primary clear agriculture road water
2,test_2,primary clear agriculture road water
3,test_3,primary clear agriculture road water
4,test_4,primary clear agriculture road water


In [19]:
submit_df.to_csv('./submission.csv')

In [ ]:
img_header = []
tags_header=[]
    
df = pd.read_csv('./submission.csv')
for c,img_name in enumerate(tqdm(df['image_name'],miniters=1000)):
        result = []
        if c >=40669:
            img = cv2.imread(f'../input/planets-dataset/test-jpg-additional/test-jpg-additional/{img_name}.jpg')
        else:
            img = cv2.imread(f'../input/planets-dataset/planet/planet/test-jpg/{img_name}.jpg')

        img = cv2.resize(img,(90,90))
        img = np.reshape(img,[1,90,90,3])
        img = img/255.0
        proba = model.predict(img)[0]
        idxs = np.argsort(proba)[::-1][:5]
        for tags,v in label_map.items():
            for i in idxs:
                if v==i:
                    result.append(tags)
        img_header.append(img_name)
        tags_header.append([' '.join(result)])
        result.clear()




 57%|█████▋    | 35000/61191 [03:56<02:59, 146.15it/s]

In [15]:
final_data = {'image_name':img_header,'tags':tags_header}
data = pd.DataFrame(final_data)
data["tags"]= data["tags"].astype(str) 
data["tags"]= data["tags"].str.slice(2, -2) 
data.head()
data.to_csv('./submitted_final.csv',index=False)